## Задача
Надо по одному состоянию клеточного автомата предсказать другое.

Обучающий файл получен таким кодом:

Для оценки решения используется `MAE`

## Данные

Данные представлены в формате `csv`.

По стокам записаны объекты

- столбец `id` - номер объекта (нумерация своя в обучающей и тестовой выборках)
- столбец `steps` - число шагов - на сколько может отличаться тест от обучения (от 1 до 5)
- столбцы с префиксом `x` - признаки, по сути они кодируют бинарную матрицу размера 22x22
- столбцы с префиксом `y` - целевые признаки, также кодируют бинарную матрицу размера 22x22

Эти две матрицы представляют собой состояния клеточного автомата на расстоянии steps шагов.

В тесте для каждого `id` дана только одна матрица - надо предсказать вторую. Обратите внимание на образец ответа - для каждого `id` - 22*22 чисел.

In [1]:
from keras.layers import Input, Dense, Conv2D, ZeroPadding2D
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import train_test_split
from tensorflow.compat.v1.keras import backend as K
from keras.models import Model

import re
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
seed_value= 42

os.environ['PYTHONHASHSEED']=str(seed_value)

random.seed(seed_value)

np.random.seed(seed_value)

tf.compat.v1.set_random_seed(seed_value)


session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

/kaggle/input/ozon-masters-2020/example_ans.csv
/kaggle/input/ozon-masters-2020/gtrain.csv
/kaggle/input/ozon-masters-2020/gtest.csv


In [2]:
train = pd.read_csv('/kaggle/input/ozon-masters-2020/gtrain.csv')
test = pd.read_csv('/kaggle/input/ozon-masters-2020/gtest.csv')

# CNN или как прийти к нему

Некие рассуждения по поводу начального решения задачи:

- группируем данные по количеству шагов `steps` и для каждого будем отдельно предсказывать положение клеточного автомата. Это необходимое условие, предсказывать всю таблицу сразу, очевидно, не имеет смысла
- очевидно, что если мы сможем угадать правила игры по первому шагу, то мы сможем предсказать положение клеточного автомата на любое количество шагов, поэтому прежде всего обращаем внимание на `steps=1`. В обычной игре "жизнь", например, возможно выучить все расположения матрицы $3 \times 3$, коих $2^9 = 512$ и по ней строить возможные модели со стопроцентной точностью. В данном случае, как окажется, мы не сможем подобрать какие-то точные правила для игры
- из предыдущего пункта также следует, что обратить внимание с самого начала стоит на точность предсказания именно на первом шагу. С него мы и начнем разбираться

# Предсказание на один шаг вперед

## Input & Output

Для начала разберемся с входными и выходными данными. На вход подается матрица, размером $22 \times 22$ и на выходе мы тоже должны получить матрицу такого же размера. Данную постановку задачи хочется свести к более понятной для машинного обучения.

В качестве оценки модели использовалась метрика `mean absolute error - MAE`:

$$ MAE = \frac{1}{n}\sum\limits_{i=1}^n \left|y_i-\hat{y}_i\right|$$

## Striding and Padding

Идея, хоть и очевидная, заключалась в следующем - давайте рассматривать все матрицы $3 \times 3$, из которых состоит входящая матрица $22 x 22$ в качестве $x$ - признаков, а в качестве $y$ - target-переменной - будем использовать то число, которое является центральным для данной матрицы. Например, пусть начальная матрица и матрица через шаг действия клеточного автомата имеет вид:

$$X = \begin{pmatrix} 
{1} & 0 & 1 & 1 & \ldots \\ 
1 & 1 & 0 & 0 & \ldots \\
0 & 1 & 0 & 0 & \ldots \\
\ldots & \ldots & \ldots & \ldots & \ldots \\
\end{pmatrix} \qquad 22 \times 22 \qquad Y = \begin{pmatrix} 
0 & 0 & 0 & 1 & \ldots \\ 
0 & 0 & 1 & 1 & \ldots \\
1 & 1 & 0 & 0 & \ldots \\
\ldots & \ldots & \ldots & \ldots & \ldots \\
\end{pmatrix} \qquad 22 \times 22$$

Тогда будут рассматриваться следующие матрицы входов и следующие target-переменные:

$$X^1 = \begin{pmatrix} 
\color{blue}{1} & \color{blue}{0} & \color{blue}{1} \\ 
\color{blue}{1} & \color{blue}{1} & \color{blue}{0} \\
\color{blue}{0} & \color{blue}{1} & \color{blue}{0} \\
\end{pmatrix} \qquad Y^1 = \begin{pmatrix} 
0 & 0 & 0 \\
0 & \color{blue}{0} & 1 \\
1 & 1 & 0 \\
\end{pmatrix}$$

$$X^2 = \begin{pmatrix} 
\color{red}{0} & \color{red}{1} & \color{red}{1} \\ 
\color{red}{1} & \color{red}{0} & \color{red}{0} \\
\color{red}{1} & \color{red}{0} & \color{red}{0} \\
\end{pmatrix} \qquad Y^2 = \begin{pmatrix} 
0 & 0 & 1 \\
0 & \color{red}{1} & 1 \\
1 & 0 & 0 \\
\end{pmatrix}$$
и так далее.

Правда, при данном преобразовании, когда мы вычленим все матрицы из начальной матрицы $22 \times 22$, матрица target-переменной будет размерности $20\times 20$. Для этого к начальной матрице $22 \times 22$ применим метод `Padding`, который грубо говоря окаймляет исходную матрицу. Данную койму матрицы можно заполнять различными способами - `zero-padding, constant-padding...` (более подробно можно почитать [здесь](https://numpy.org/doc/stable/reference/generated/numpy.pad.html)). Я использовал методом проб и ошибок два способа - нулевой padding - мы предполагаем, что, на краях у нас больше все-таки вероятность умереть в клеточном автомате и `wrap-padding` - мы предполагаем мысленно, что наша игра является замкнутой- если бы мы смогли обернуть шар матрицей игры, то мы бы левым верхним углом взаимодействовали с правым нижним (это просто нужно представить и осознать). 

В результате данного преобразовани мы получим матрицу размером $X = 24 \times 24$, из которой можно уже последовательно забирать матрицы $3 \times 3$ , тогда и преобразованный вектор $X'$ и вектор предсказаний $Y'$ для каждой такой матрицы будет иметь размерность $22 \times 22$. 

Так, например, входная матрица для задач машинного обучения в данном случае будет размера 

$$X.shape = (L \times 22 \times 22, 9)$$ 

где $L$ - количество элементов в обучающей выборке;

$$Y.shape = (L \times 22 \times 22, 1)$$

Соответственно, мы можем применять ML-модели, предназначенные для бинарной классификации над вектором признаков.

## ML models

1. Первая модель, которая использовалась в качестве Baseline, да и вообще первое, что приходит на ум - `Logistic Regression` - будем узнавать вероятность того, что объект принадлежит одному из двух классов $Y \in \{0,1\}$. Использовалась `Logloss` функция потерь для обучения (бинарная кроссентропия): $$\operatorname{Logloss} = - \ln \prod\limits_{x \in 1} p_x - \ln \prod\limits_{x \in 0} (1-p_x) \to \min$$ где  $$p_x = \sigma(f(x)) \qquad \sigma(x) = \frac{1}{1+e^{-x}}$$ вероятность того, что объект принадлежит классу $1$. Данная модель имела преимущество по времени работы - примерно $15$ секунд на тренировку и предсказание и после кросс-валидации `MAE = 0.315`. 

2. После этого были созданы попытки бинарной классификации с помощью `RandomForestClassifier` и использование стэккинга моделей, но модели не отличались от логистической регрессии по качеству, а по затрачиваемому времени - обучались дольше в разы.

3. Осознание, почему при $512$ состояних матрицы $3 \times 3$ мы угадываем следующее состояние с вероятностью $\frac{1}{3}$, пришло позже - действительно, данный клеточный автомат содержит в себе какие-то другие закономерности, которые могут не подаются простому заучиванию правил - положение в следующий момент времени может зависеть от местоположения матрицы $3\times3$ (центр или окраины). В своем мозгу я начал представлять данную задачу как моделирование заражений, в которой человек может быть заражен долго ($5$ шагов) и его выздоровление (заражение), возможно, не зависит от положения или количества соседей. Поэтому, для обнаружения более глубоких взаимодействий я начал использовать `Convolutional Neural Network` - сверточные нейронные сети. 

## CNN для предсказания на один шаг

Во-первых, почему именно свертки, а не, например, `Multilayer perceptron`. В многослойном перцептроне мы подаем на вход некоторый вектор признаков и предполагаем, что все элементы в нем `линейно независимы`. Из-за этого у нас и возможно перемножение данного вектора на матрицу весов каждого слоя. Но в данной задаче мы не можем говорить о независимости входящих признаков - в клеточном автомате ячейки связаны с $9$ ячейками и о независимости речь не идет, следовательно информация клеточного автомата зависит от каждого отдельной ячейки.

Предположим, что нас волнует `свойство локальности` - клетки упорядочены друг с другом и нас волнуют только те клетки, которые рядом (это предположение мы не немного усилим на `steps !=1`).

Поэтому я использовал сверточную сеть со следующей архитектурой:

- на вход подавался тензор с $(L,22,22,9)$ - каждый элемент матрицы $22 \times 22$ являлся вектором из $9$ элементов 
    - после этого применялся сверточный слой, состоящий из $K$ фильтров размером $3 \times 3$, `valid` - логика понятна, мы пытаемся понять некоторые закономерности из данного вектора. Выход слоя: $(L,20,20,K)$
    - после этого применяется `ZeroPadding(1)` - операция, аналогичная рассмотренной выше. Выход слоя: $(L,22,22,K)$
- данную конструкцию выше я использолал $3$ раза для попытки более точно понять структуру модели
- на выходе использовался $1$ нейрон с функцией активации `sigmoid`. Мы получаем вероятность принадлежности единице ($(L,22,22,1)$.

Получилось, что данная модель работала гораздо лучше, чем `Logistic Regression` - благодаря тому, что Receptive Field нейрона - поле видимости - спустя $3$ слоя свертки повышался до $49$ нейронов, сеть улавливала некоторые особенности не только матриц $3\times 3$, но и $7 \times 7$.

Данная модель обучается гораздо дольше (20 минут на 3 эпохах), но зато она

- дает результат лучше, чем модель машинного обучения - $MAE \approx 0.26$
- не склонна к переобучению, ведь по сути она выучиывает правила игры для внутренних матриц $7\times7$.

# Предсказание на более, чем один шаг

Осознание того, что модель дает $\frac{1}{4}$ вероятность ошибки, не позволяет нам предсказывать состояние `steps==2` и дообучать модель на данных, предназначенных для второго шага. Даже сейчас я смутно представляю, как возможно предсказать нормально хотя бы на два шага вперед. Но была предпринята попытка и предложена идея - будем использовать фильтр не размером $3\times3$ - при предсказании начальных правил не хватит, чтобы понять поведение на следующих шагах, а $7\times7$ - так модель будет пытаться рассматривать что-то в далеке от клетки - RF нейрона увеличится, а так как модель построена из малого количества сверточных слоев, то мы можем использовать такой большой фильтр. 

Идея сработала - сверточная нейронная сеть стала предсказывать лучше (судя по метрики MAE) в среднем на сотую. Итого, средние оценки для каждого количества шагов:

|steps | MAE  |
|------|------|
|   1  | 0.261|
|   2  | 0.355|
|   3  | 0.37 |
|   4  | 0.375|
|   5  | 0.384|
|-|$$\operatorname{MAE_{mean}} \approx 0.35$$|

что и отражается (пока что) в лидерборде.

# Заключение

Для реализации данных идей был написан класс `CnnCellullarAutomata` с подробной документацией, для построения модели сверточных сетей использовался `Keras`.

Мне было интересно участвовать в соревновании, первом соревновании, в котором свои идеи я смог довести до конца и меня это очень радует. Особо радует, что я навязал конкуренцию сильным конкурентам. Жду результатов на `private-leaderbord` и жду разбора задачи от организаторов - очень хочется понять реализацию предсказания на несколько шагов вперед в соревновании по машинному обучению. Говорят, что можно и без него здесь обойтись. Что ж, я не смог :)

Реализация приведена ниже. Спасибо!

In [3]:
class CnnCellullarAutomata:
    """
    Class : CNN Prediction For Cellular Automata

    ....................^_^.....................

    Attributes
    ----------
    step : int
        'steps' in input Dataframe, amount of 'life_steps' between 'steps' amount of moments in the game
        
    train : DataFrame
        train dataset
        
    test : Dataframe
        test dataset
        
    data : DataFrame
        particular 'steps' data
        
    n, defalut = 3: int (odd)
        shape of filter matrix nxn 
        
    shape : int
        shape of input matrix of CA
        
    lists, defalut = 200 :
        amout of filters in every Convolutional Layer
        
    mode, default = 'wrap':
        np.pad padding mode
        
    ....................^_^.....................
        
    Methods
    -------
    reshape_to_cnn(matrix: np.array) -> np.array: 
        reshape matrix to convolutional neural network input
        
    wrap_padding(self,matrix: np.array, mode: str) -> np.array:
        make padding for input matrix
        
    find_columns(param: str): -> np.array:
        return np.array 'x_%' and 'y_%' columns
        
    get_data(variable: str): -> np.array:
        return np.array 'x_%' and 'y_%' columns || wrap_padding
    
    all_3x3(matrix: np.array) -> np.array:
        return all 3x3 matrix of nxn array
        
    preprocess() -> np.array:
        reshaping data
        
    fit(epochs: int = 3) -> model:
        return fitted keras cnn model
        
    result(epochs: int) -> np.array:
        return predict answer
        
    
    """
    def __init__(self, step, train, test, n=3, lists=200, mode = 'wrap'):
        
        self.step = step 
        self.train = train
        self.test = test
        self.data = train[train['steps']== step]
        self.n = n
        self.shape = 22
        self.mode = mode
        self.lists = lists
        
    def reshape_to_cnn(matrix: np.array):
        
        return matrix.reshape(matrix.shape[0], matrix.shape[1], matrix.shape[2], 1)

    def wrap_padding(self,matrix, mode):
        
        w = int((self.n-0.5*2)/ 2)
        return np.array([np.pad(m, (w,w), mode=mode) for m in matrix])
        
    def find_columns(self,param):
        
        input_columns = self.train.columns.values
        
        reg_exp = re.compile(f'{param}_.+')
        
        match_exp = np.vectorize(
            lambda x:bool( reg_exp.match(x) ) )
        
        match_columns = match_exp(input_columns)
        output_columns = input_columns[match_columns]
        return output_columns
        
    def get_data(self,variable):
    
        column = CnnCellullarAutomata.find_columns(self,variable)
        
        return self.data[column].values
    

    def all_3x3(self, matrix):
            
        i, j = 1 + (matrix.shape[0] - self.n), 1 + (matrix.shape[1] - self.n) 
        all_matrix = np.lib.stride_tricks.as_strided(matrix, shape = (i ,j, self.n, self.n), strides = matrix.strides + matrix.strides)
        
        
        return all_matrix
        
    def preprocess(self):
        
        x_data = CnnCellullarAutomata.get_data(self, variable='x').reshape(-1,self.shape,self.shape)
        x_data = CnnCellullarAutomata.wrap_padding(self,x_data,self.mode)
        x_data = np.array(list(map(lambda x: CnnCellullarAutomata.all_3x3(self, x),x_data))).reshape((-1,self.shape,self.shape,self.n*self.n))

        y_data = CnnCellullarAutomata.get_data(self, variable='y').reshape(-1,self.shape,self.shape,1)
        
        return x_data, y_data

    def fit(self,epochs = 3):
        
        x_data,y_data = CnnCellullarAutomata.preprocess(self)
        #X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
        w = int((self.n-0.5*2)/ 2)
    
        input = Input(shape=(self.shape, self.shape, self.n*self.n))

        X = Conv2D(self.lists, (self.n,self.n), padding='valid', activation='relu', strides=1)(input)
        X = ZeroPadding2D(w)(X)
        X = Conv2D(self.lists, (self.n,self.n), padding='same', activation='relu', strides=1)(X)
        X = ZeroPadding2D(w)(X)
        X = Conv2D(self.lists, (self.n,self.n), padding='valid', activation='relu', strides=1)(X)
        X = ZeroPadding2D(w)(X)
        X = Conv2D(self.lists, (self.n,self.n), padding='valid', activation='relu', strides=1)(X)
        X = Dense(32,activation='relu')(X)
        X = Dense(1,activation='sigmoid')(X)

        model = Model(inputs=input,outputs = X)

        model.summary()
        
        model.compile('adam', "binary_crossentropy", metrics=["mean_absolute_error"])
        #         run_hist_1 = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=epochs)
        model.fit(x_data, y_data, epochs = epochs)
        #print(mae(model.predict(X_test).round().reshape(-1,self.shape*self.shape),y_test.reshape(-1,self.shape*self.shape)))
        return model
    
    def result(self,epochs):
        model = CnnCellullarAutomata.fit(self,epochs)
        
        result = self.test[self.test['steps']==self.step][CnnCellullarAutomata.find_columns(self, 'x')].values
        result = result.reshape(-1,self.shape,self.shape)
        result = CnnCellullarAutomata.wrap_padding(self,result,self.mode)
        result = np.array(list(map(lambda x: CnnCellullarAutomata.all_3x3(self, x),result))).reshape((-1,self.shape,self.shape,self.n*self.n))
        output = model.predict(result).round().reshape(-1,self.shape*self.shape)
        return output

## Training

In [4]:
d = {'step':[1,2,3,4,5],'lists':[200,200,32,32,32],'epochs':[2,2,3,3,3],'n':[3,3,7,7,7]}

for x_1,x_2,x_3,x_4 in zip(*d.values()):
    r = CnnCellullarAutomata(step=x_1,train=train,test=test,lists=x_2,n=x_4)
    exec("y_pred_step_%s = r.result(epochs=%s)" % (x_1,x_3))
    print(f"{x_1} step is over")

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 22, 22, 9)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 200)       16400     
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 22, 22, 200)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 200)       360200    
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 24, 24, 200)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 200)       360200    
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 24, 24, 200)       0     

## Submission

In [5]:
def find_columns(train, param):

    r = re.compile(f'{param}_.+')
    vmatch = np.vectorize(lambda x:bool(r.match(x)))
    sel = vmatch(train.columns.values)
    return train.columns.values[sel]

x_columns ,y_columns = find_columns(train,'x'), find_columns(train,'y')

id_1_test = test[test['steps']==1][x_columns].index.values
id_2_test = test[test['steps']==2][x_columns].index.values
id_3_test = test[test['steps']==3][x_columns].index.values
id_4_test = test[test['steps']==4][x_columns].index.values
id_5_test = test[test['steps']==5][x_columns].index.values

answer_columns = np.append('id',y_columns)
answer_index  = np.arange(0, test.shape[0])

submission = pd.DataFrame(columns = answer_columns, index = answer_index)
submission['id'] = test['id']

submission.loc[id_1_test,y_columns] = y_pred_step_1
submission.loc[id_2_test,y_columns] = y_pred_step_2
submission.loc[id_3_test,y_columns] = y_pred_step_3
submission.loc[id_4_test,y_columns] = y_pred_step_4
submission.loc[id_5_test,y_columns] = y_pred_step_5

submission.to_csv('submission.csv', index = False)
print('done')

done
